In [1]:
import torch
import torch.nn as nn

In [2]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.constants import POSITION_COLS, MASS_CLASS_COLS, BASIC_TRAINING_COLS, FORCE_CLASS_COLS
from isaac.sanity import class_proportions
from isaac.models import initialise_model
from isaac.training import training_loop
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

directory = "GRU_singlebranch/"
create_directory(directory)

cuda:0


In [4]:
TR_COLS = BASIC_TRAINING_COLS
NORMALISE_DATA = True
BATCH_SIZE = 128
EPOCHS = 100
STEP_SIZE = 1
SEQ_END = 2700

In [5]:
train_trials = read_dataset("data/train_passive_trials.h5", n_trials=3500, cols=TR_COLS)
val_trials = read_dataset("data/val_passive_trials.h5", n_trials=1000, cols=TR_COLS)

100%|██████████| 1000/1000 [00:09<00:00, 108.69it/s]


In [6]:
INPUT_DIM = len(TR_COLS)    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

network_params = (INPUT_DIM, HIDDEN_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)

# MASS TRAINING

In [7]:
loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=MASS_CLASS_COLS, 
                                  training_columns=TR_COLS, batch_size=BATCH_SIZE, 
                                  normalise_data=NORMALISE_DATA, device=device)

100%|██████████| 1000/1000 [00:00<00:00, 1083.39it/s]


In [8]:
train_loader, val_loader = loaders

In [9]:
class_proportions(train_loader)
class_proportions(val_loader)

[1188 1174 1138]
Majority class:  0.3394285714285714
[340 349 311]
Majority class:  0.349


(array([340, 349, 311]), 0.349)

In [10]:
N_MODELS = 3

stats_dfs = []
directory = "models/GRU_singlebranch/"
create_directory(directory)
    
for seed in range(N_MODELS):
    df = pd.DataFrame(columns=["seed", "Epoch", "Loss"])

    model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device)
    epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, train_loader, 
                                                               val_loader, EPOCHS, seq_end=SEQ_END,
                                                               step_size=STEP_SIZE)

    torch.save(best_model.state_dict(), directory + "best_mass_model_seed_%d.pt" % seed)

    df["Epoch"] = np.arange(EPOCHS)
    df["Loss"] = epoch_losses
    df["Train Accuracy"] = epoch_accuracies[0]
    df["Val Accuracy"] = epoch_accuracies[1]
    df["seed"] = seed
    stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(directory+"mass_stats.h5", key="stats")

Train_loss: ([0.76235775]) Train_acc: (58.0) Val_acc: ([53.5]): 100%|██████████| 100/100 [52:40<00:00, 32.01s/it]             
Train_loss: ([0.59266168]) Train_acc: (65.51428571428572) Val_acc: ([56.8]): 100%|██████████| 100/100 [53:41<00:00, 32.31s/it]
Train_loss: ([0.52249955]) Train_acc: (66.54285714285714) Val_acc: ([55.7]): 100%|██████████| 100/100 [53:38<00:00, 31.89s/it]


# FORCE TRAINING

In [11]:
loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=FORCE_CLASS_COLS, 
                                  training_columns=TR_COLS, batch_size=BATCH_SIZE, 
                                  normalise_data=NORMALISE_DATA, device=device)

100%|██████████| 1000/1000 [00:01<00:00, 981.62it/s]


In [12]:
train_loader, val_loader = loaders

In [13]:
class_proportions(train_loader)
class_proportions(val_loader)

[1183 1169 1148]
Majority class:  0.338
[337 350 313]
Majority class:  0.35


(array([337, 350, 313]), 0.35)

In [14]:
N_MODELS = 3

stats_dfs = []
directory = "models/GRU_singlebranch/"
create_directory(directory)
    
for seed in range(N_MODELS):
    df = pd.DataFrame(columns=["seed", "Epoch", "Loss"])

    model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device)
    epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, train_loader, 
                                                               val_loader, EPOCHS, seq_end=SEQ_END,
                                                               step_size=STEP_SIZE)

    torch.save(best_model.state_dict(), directory + "best_force_model_seed_%d.pt" % seed)

    df["Epoch"] = np.arange(EPOCHS)
    df["Loss"] = epoch_losses
    df["Train Accuracy"] = epoch_accuracies[0]
    df["Val Accuracy"] = epoch_accuracies[1]
    df["seed"] = seed
    stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(directory+"force_stats.h5", key="stats")

Train_loss: ([0.48224055]) Train_acc: (64.0) Val_acc: ([60.2]): 100%|██████████| 100/100 [53:23<00:00, 31.80s/it]             
Train_loss: ([0.33556286]) Train_acc: (67.22857142857143) Val_acc: ([52.0]): 100%|██████████| 100/100 [52:47<00:00, 32.00s/it]
Train_loss: ([0.73122307]) Train_acc: (53.0) Val_acc: ([46.5]): 100%|██████████| 100/100 [53:26<00:00, 32.46s/it]             
